In [ ]:
import cv2
import numpy as np

# --- Variables Globales ---
drawing_mode = 'rectangle' # Modos: 'rectangle', 'circle', 'line', 'ellipse', 'text'
start_point = (-1, -1)
is_drawing = False
text_entry_pos = None

# Historial de cambios para la función de deshacer
history = []

# --- Función de Callback del Mouse ---
def draw_shape(event, x, y, flags, param):
    global start_point, is_drawing, canvas, drawing_mode, text_entry_pos

    # Si estamos en modo texto, solo registramos el clic y no dibujamos arrastrando.
    if drawing_mode == 'text':
        if event == cv2.EVENT_LBUTTONDOWN:
            # Guardamos el estado actual antes de prepararnos para añadir texto
            history.append(canvas.copy())
            text_entry_pos = (x, y) # Guardamos la posición para el bucle principal
        return # Salimos para no ejecutar el resto de la lógica de dibujo

    # Lógica para dibujar figuras geométricas
    if event == cv2.EVENT_LBUTTONDOWN:
        is_drawing = True
        start_point = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE:
        if is_drawing:
            temp_canvas = canvas.copy()
            if drawing_mode == 'rectangle':
                # Dibuja un rectángulo 
                cv2.rectangle(temp_canvas, pt1=start_point, pt2=(x, y), color=(0, 255, 0), thickness=2)
            elif drawing_mode == 'line':
                 # Dibuja una línea 
                cv2.line(temp_canvas, pt1=start_point, pt2=(x,y), color=(0,0,255), thickness=2)
            elif drawing_mode == 'circle':
                 # Dibuja un círculo 
                radius = int(np.sqrt((x - start_point[0])**2 + (y - start_point[1])**2))
                cv2.circle(temp_canvas, center=start_point, radius=radius, color=(255, 0, 0), thickness=2)
            elif drawing_mode == 'ellipse':
                # Dibuja una elipse 
                center_x = (start_point[0] + x) // 2
                center_y = (start_point[1] + y) // 2
                axis1_len = abs(start_point[0] - x) // 2
                axis2_len = abs(start_point[1] - y) // 2
                cv2.ellipse(temp_canvas, (center_x, center_y), (axis1_len, axis2_len), 0, 0, 360, (255, 255, 0), 2)

            cv2.imshow('Dibujo Interactivo', temp_canvas)

    elif event == cv2.EVENT_LBUTTONUP:
        is_drawing = False
        history.append(canvas.copy())
        
        if drawing_mode == 'rectangle':
            cv2.rectangle(canvas, pt1=start_point, pt2=(x, y), color=(0, 255, 0), thickness=2)
        elif drawing_mode == 'line':
            cv2.line(canvas, pt1=start_point, pt2=(x,y), color=(0,0,255), thickness=2)
        elif drawing_mode == 'circle':
            radius = int(np.sqrt((x - start_point[0])**2 + (y - start_point[1])**2))
            cv2.circle(canvas, center=start_point, radius=radius, color=(255, 0, 0), thickness=2)
        elif drawing_mode == 'ellipse':
            center_x = (start_point[0] + x) // 2
            center_y = (start_point[1] + y) // 2
            axis1_len = abs(start_point[0] - x) // 2
            axis2_len = abs(start_point[1] - y) // 2
            cv2.ellipse(canvas, (center_x, center_y), (axis1_len, axis2_len), 0, 0, 360, (255, 255, 0), 2)


# --- Creación del Lienzo ---
canvas = np.full((600, 800, 3), 255, dtype=np.uint8)
history.append(canvas.copy())

cv2.namedWindow('Dibujo Interactivo')
cv2.setMouseCallback('Dibujo Interactivo', draw_shape)

# --- Impresión de Instrucciones ---
print("Programa de Dibujo Interactivo Mejorado")
print("---------------------------------------")
print("Figuras: 'r' (rectángulo), 'c' (círculo), 'l' (línea), 'e' (elipse)")
print("Texto: 't' (activa el modo texto, haz clic y escribe en la consola)")
print("Opciones: 'u' (deshacer), 's' (guardar), 'q' (salir)")

# --- Bucle Principal ---
while True:
    cv2.imshow('Dibujo Interactivo', canvas)
    
    # Manejo de la entrada de texto fuera del callback para no bloquear la ventana
    if text_entry_pos:
        text = input(f"Escribe el texto a insertar en la posición {text_entry_pos}: ")
        if text:
            # Se definen los argumentos para escribir texto 
            font = cv2.FONT_HERSHEY_SIMPLEX # 
            # Se escribe el texto en la imagen 
            cv2.putText(canvas, text=text, org=text_entry_pos, fontFace=font, fontScale=1, color=(0, 0, 0), thickness=2, lineType=cv2.LINE_AA)
        else:
            # Si el usuario no escribe nada, revertimos el guardado en el historial
            history.pop()
        text_entry_pos = None # Restablecemos la variable

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('r'):
        drawing_mode = 'rectangle'
        print("Modo actual: Rectángulo")
    elif key == ord('c'):
        drawing_mode = 'circle'
        print("Modo actual: Círculo")
    elif key == ord('l'):
        drawing_mode = 'line'
        print("Modo actual: Línea")
    elif key == ord('e'):
        drawing_mode = 'ellipse'
        print("Modo actual: Elipse")
    elif key == ord('t'):
        drawing_mode = 'text'
        print("Modo actual: Texto (haz clic en el lienzo y escribe en la consola)")
    elif key == ord('s'):
        # Guarda la imagen en el directorio 
        cv2.imwrite('mi_dibujo_mejorado.png', canvas)
        print("¡Dibujo guardado como 'mi_dibujo_mejorado.png'!")
    elif key == ord('u'):
        if len(history) > 1:
            history.pop()
            canvas = history[-1].copy()
            print("Última acción deshecha.")
        else:
            print("No hay más acciones que deshacer.")

# Cierra todas las ventanas de OpenCV
cv2.destroyAllWindows()